In [84]:
import wfc
from wfc import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from lasagna.imports import *
import lasagna.io
j = lasagna.config.j = start_client()
dr = ((0, 1),)*100
def show(*args, **kwargs):
    lasagna.io.show_IJ(*args, display_ranges=dr, **kwargs)

In [131]:
N = 3
FMX = N*9
FMY = N*9

a = (1, 0, 1)
b = (0, 1, 0)
c = (1, 1, 1)
d = (0, 0, 0)

T = rotate((c,b,b), 4)
X = rotate((a,b,a), 1)
I = rotate((c,b,c), 2)


initial = [(a,b,a), (b,a,b), (b,b,b), (a,a,a)]
initial = T

filename = 'samples/Knot.png'
initial = initial_from_png(filename, N);

model = SimpleTiledModel(N,FMX,FMY,initial)
print 'initial has %d unique tiles' % len(model.initial)
arr = []

initial has 49 unique tiles


In [86]:
model.wave = np.random.choice([True, False], p=[0.9,0.1], size=model.wave.shape)
# model.wave[0,:,:] = True

In [89]:
%timeit -n1 -r1 model.propagate_from(zip_where(model.wave.sum(axis=0)))

propagation modified 2808 points, touched 27036
1 loop, best of 1: 2.56 s per loop


In [99]:
%prun -s cumulative model.propagate_from(zip_where(model.wave.sum(axis=0)))

propagation modified 0 points, touched 5512
 

In [132]:
%prun -s cumulative do_it()

collapsed (16, 4)
propagation deleted 6486 coefficients in 2863 points; touched 21822 points
collapsed (15, 6)
propagation deleted 1256 coefficients in 304 points; touched 2369 points
collapsed (13, 6)
propagation deleted 191 coefficients in 78 points; touched 558 points
collapsed (14, 4)
propagation deleted 96 coefficients in 43 points; touched 287 points
collapsed (15, 8)
propagation deleted 274 coefficients in 130 points; touched 1034 points
collapsed (13, 5)
propagation deleted 721 coefficients in 168 points; touched 1247 points
collapsed (13, 8)
propagation deleted 1702 coefficients in 409 points; touched 3139 points
collapsed (11, 10)
propagation deleted 78 coefficients in 44 points; touched 342 points
collapsed (14, 2)
propagation deleted 23 coefficients in 16 points; touched 105 points
collapsed (11, 4)
propagation deleted 129 coefficients in 60 points; touched 443 points
collapsed (16, 8)
propagation deleted 207 coefficients in 101 points; touched 791 points
collapsed (10, 7)


In [130]:
def do_it():
    for i in range(200):
        done = model.update()
        if done:
            break
    #     arr += [model.get_superposition()]
    print i

In [123]:
model.wave.sum(axis=0)

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1],
       [1, 1, 1, 1, 

In [388]:
arr = np.array(arr)
skimage.external.tifffile.imsave('Platformer_run.tif', arr.astype(np.float32))

In [118]:
show(model.get_superposition(), 'superposition')
# show(model.get_entropy(), 'entropy')
show(model.wave.sum(axis=0), 'wavesum')
# show(model.initial, 'initial')

Cast float64 to float32
Cast int64 to int16 without loss
